# Hill Climbing — TSP (10 ciudades)

Notebook inspirado en el estilo de implementación de **Hill Climbing** usado en N-Reinas:
- Separación clara de **estado**, **costo**, **vecindario** y **algoritmo**.
- Versión *best-improvement* por defecto; incluye opción *first-improvement* y **sideways moves** (para mesetas).
- Dos vecindarios clásicos para TSP: **Swap-2** y **2-opt**.


## Objetivo
Minimizar la longitud del tour (ciclo cerrado) que visita una sola vez las 10 ciudades y regresa al origen.

La matriz de distancias `D` está indexada de 0..9 (ciudades 1..10).

In [ ]:

import random, math
from typing import List, Tuple
import pandas as pd
import matplotlib.pyplot as plt

# --- Matriz de distancias (índices 0..9 corresponden a ciudades 1..10) ---
D = [
 [0,120,250,310,180,200,270,150,340,290],
 [120,0,190,220,270,150,280,200,300,250],
 [250,190,0,110,160,210,140,180, 90,220],
 [310,220,110,0,140,180,130,150,100,200],
 [180,270,160,140,0,190,170,210,120,240],
 [200,150,210,180,190,0,160,120,230,210],
 [270,280,140,130,170,160,0, 90,150,100],
 [150,200,180,150,210,120, 90,0, 180,160],
 [340,300, 90,100,120,230,150,180,0, 170],
 [290,250,220,200,240,210,100,160,170,0 ],
]

def tour_length(route: List[int], dist=D) -> int:
    """Longitud del tour (ciclo cerrado). route es permutación de 0..n-1."""
    s = 0
    for i in range(len(route)):
        a = route[i]
        b = route[(i+1) % len(route)]
        s += dist[a][b]
    return s

def pretty(route: List[int]) -> str:
    """Imprime la ruta 1-basada cerrando el ciclo."""
    return " -> ".join(str(x+1) for x in route + [route[0]])


In [ ]:

# --- Generadores de solución inicial ---
def random_route(n: int) -> List[int]:
    r = list(range(n))
    random.shuffle(r)
    return r

def greedy_route(start: int = 0, dist=D) -> List[int]:
    """Construye una ruta avara (nearest-neighbor) comenzando en 'start'."""
    n = len(dist)
    unvisited = set(range(n)); unvisited.remove(start)
    route = [start]
    while unvisited:
        last = route[-1]
        nxt = min(unvisited, key=lambda j: dist[last][j])
        unvisited.remove(nxt)
        route.append(nxt)
    return route


In [ ]:

# --- Vecindarios ---
def neighbors_swap2(route: List[int]):
    """Intercambia dos posiciones (Swap-2)."""
    n = len(route)
    for i in range(n-1):
        for j in range(i+1, n):
            nr = route[:]
            nr[i], nr[j] = nr[j], nr[i]
            yield nr

def neighbors_2opt(route: List[int]):
    """Revierte el segmento [i:j] (2-opt)."""
    n = len(route)
    for i in range(n-1):
        for j in range(i+1, n):
            nr = route[:i] + list(reversed(route[i:j+1])) + route[j+1:]
            yield nr


In [ ]:

# --- Hill Climbing ---
def hill_climb(
    init_route: List[int],
    dist=D,
    neighborhood: str = "2opt",   # "2opt" o "swap"
    strategy: str = "best",       # "best" o "first"
    sideways_limit: int = 0,      # permite moverse en mesetas (igual costo) hasta K veces
    max_iters: int = 10000,
    verbose: bool = False
) -> Tuple[List[int], int, list]:
    """Devuelve (mejor_ruta, mejor_costo, historial_costos)."""
    neigh_fn = neighbors_2opt if neighborhood.lower() == "2opt" else neighbors_swap2
    curr = init_route[:]
    curr_cost = tour_length(curr, dist)
    history = [curr_cost]
    sideways = 0

    for it in range(max_iters):
        improved = False
        best_neighbor = None
        best_cost = curr_cost

        neighs = list(neigh_fn(curr))
        if strategy.lower().startswith("first"):
            random.shuffle(neighs)

        for nb in neighs:
            c = tour_length(nb, dist)
            if c < curr_cost:
                if strategy.lower().startswith("first"):
                    curr, curr_cost = nb, c
                    improved = True
                    sideways = 0
                    break
                else:  # best-improvement
                    if c < best_cost:
                        best_neighbor, best_cost = nb, c
            elif c == curr_cost and sideways < sideways_limit:
                # permite avanzar sobre mesetas
                curr, curr_cost = nb, c
                improved = True
                sideways += 1
                break

        if strategy.lower().startswith("best") and best_cost < curr_cost:
            curr, curr_cost = best_neighbor, best_cost
            improved = True
            sideways = 0

        history.append(curr_cost)
        if verbose:
            print(f"Iter {it+1:>4}  costo={curr_cost}")

        if not improved:
            break

    return curr, curr_cost, history


## Demostración rápida (una corrida)

In [ ]:

random.seed(42)

# Elige ruta inicial (cambia entre aleatoria y avara para comparar)
init = random_route(len(D))
# init = greedy_route(start=0)

best_route, best_cost, hist = hill_climb(
    init_route=init,
    neighborhood="2opt",  # prueba también "swap"
    strategy="best",      # prueba también "first"
    sideways_limit=0,
    verbose=False
)

print("Ruta inicial: ", pretty(init), "| costo =", tour_length(init))
print("Mejor ruta:   ", pretty(best_route), "| costo =", best_cost)

# Trazo del costo vs iteración
plt.figure()
plt.plot(range(len(hist)), hist)
plt.xlabel("Iteración")
plt.ylabel("Costo del tour")
plt.title("Hill Climbing — trayectoria de una corrida")
plt.show()


## Reinicios aleatorios y comparación de configuraciones

In [ ]:

def multi_restart(restarts=20, neigh="2opt", strat="best", sideways_limit=0, seed=123):
    random.seed(seed)
    rows = []
    for r in range(restarts):
        init = random_route(len(D))
        br, bc, hist = hill_climb(init, D, neighborhood=neigh, strategy=strat, sideways_limit=sideways_limit)
        rows.append({"restart": r+1, "best_cost": bc, "iters": len(hist)-1})
    return pd.DataFrame(rows)

# Ejemplo: comparar vecindarios y estrategias
df_2opt_best = multi_restart(30, neigh="2opt", strat="best", seed=7)
df_swap_best = multi_restart(30, neigh="swap", strat="best", seed=7)

print("Resumen 2-opt (best):  min =", df_2opt_best.best_cost.min(), 
      " mean =", round(df_2opt_best.best_cost.mean(),2), 
      " std =", round(df_2opt_best.best_cost.std(ddof=1),2))

print("Resumen Swap-2 (best): min =", df_swap_best.best_cost.min(), 
      " mean =", round(df_swap_best.best_cost.mean(),2), 
      " std =", round(df_swap_best.best_cost.std(ddof=1),2))

# Muestra las primeras filas
df_2opt_best.head(), df_swap_best.head()


## Sugerencias para el reporte
- **Cómo generaste la solución inicial** (aleatoria vs avara) y por qué.
- **Función de vecindad**: Swap-2 vs 2-opt (ventajas/desventajas).
- **Estrategia**: best vs first y efecto de `sideways_limit`.
- **Resultados**: mejor costo, promedio, desviación; traza de costo por iteración.
- **Conclusión**: cuándo te funcionó bien HC y cuándo no; cómo ayudan los reinicios.